# Agenda, day 3 — real-world data

1. Recap + Q&A
2. More on CSV
3. Reading online data
4. Sorting
5. Grouping
6. Pivot tables
7. Joining tables
8. Cleaning data

# Recap

1. Most work in Pandas is done in a data frame
    - 2D table
    - Columns -- names must be unique
    - Rows -- with an index that doesn't need to be unique
2. We can retrieve (or assign) to a Pandas data frame using `.loc` and `.iloc`
3. In particular, using `.loc` is a key part of working with Pandas
    - In the one-argument version, we just pass a *row selector*, aka `df.loc[ROW_SELECTOR]`, which can be:
        - A string (for the index of the row(s) we want)
        - A list of strings (for the indexes of the row(s) we want)
        - A boolean series (indicating which rows we want, wherever there's a `True` in there)
        - A slice, for a number of rows
    - In the two-argument version, we pass both a *row selector* and a *column selector*, separated by a comma, which looks like: `df.loc[ROW_SELECTOR, COLUMN SELECTOR]`. The row selector is the same as the one-argument version, but the column selector is also the same, and can be used to describe which columns we want. 
4. At the end of last week, we saw that we can read data from a CSV file    

# Reading from CSV files

"CSV" stands for "comma-separated values" or "character-separated values."  The idea is:

- Every row in the file represents one record
- The field separator (a comma, by default) separates fields from one another in each record.

You can imagine a CSV file looking something like this:

```
United States,English
United Kingdom,English
France,French
Germany,German
Netherlands,Dutch
```

If the above is a CSV file, then we want to read it into a data frame, such that the country will be one column and the language will be a second column.

There is a problem with what I did here: There is no header line, giving the column names! If we aren't careful, then when we read it in, the first line will be seen as the column names, not as a data row.

Let's consider this, then:

```
country,language
United States,English
United Kingdom,English
France,French
Germany,German
Netherlands,Dutch
```

If we want to read a CSV file into a data frame, we use `pd.read_csv`. This function takes a filename as an argument, and assumes a lot of defaults for *many* options that can be passed.  Here are a few of the options we might want to pass to `pd.read_csv`.

(Note that all of these are passed as keyword arguments, meaning that they are of the form `name=value` in the function call.)

1. `sep` -- pass this to indicate what character is separating the fields on each line. By default, it's `','`. I often use `'\t'` (tab) for a separator, because I find it easier to read.
2. `header` -- pass this to indicate on what line of the file the header is located. This is useful if you want to skip over several rows, until you get to the column names or the data. If there are no column names (i.e., no header), then just pass `header=None`.
3. `usecols` -- this argument takes a list of strings (or of integers, if you really want), indicating which fields should be included in the data frame. If you pass strings, then they should be column names. If you pass integers, they should be column indexes, starting with 0.
4. `dtype` -- this argument should be a dictionary of column names and types that we want to assign. This way, you can force Pandas's hands.

In [10]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [8]:
!ls *.csv

burrito_current.csv	   languages.csv  titanic3.csv
celebrity_deaths_2016.csv  taxi.csv


In [9]:
df = pd.read_csv('taxi.csv')

In [3]:
!head taxi.csv

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30


In [5]:
df = pd.read_csv('taxi.csv', usecols=['trip_distance', 'total_amount', 'passenger_count'])

In [6]:
df.head()

,passenger_count,trip_distance,total_amount
0,1,1.63,17.80
1,1,0.46,8.30
2,1,0.87,11.00
3,1,2.13,17.16
4,1,1.40,10.30


In [7]:
df.dtypes  # what dtype is each column?

passenger_count      int64
trip_distance      float64
total_amount       float64
dtype: object

In [11]:
df = pd.read_csv('taxi.csv', 
                 usecols=['trip_distance', 'total_amount', 'passenger_count'],
                dtype={'trip_distance':np.float64,
                      'total_amount':np.float64,
                      'passenger_count':np.int8})

In [12]:
df.dtypes

passenger_count       int8
trip_distance      float64
total_amount       float64
dtype: object

In [13]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers.readers:

read_csv(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', *, sep: 'str | None | lib.NoDefault' = <no_default>, delimiter: 'str | None | lib.NoDefault' = None, header: "int | Sequence[int] | None | Literal['infer']" = 'infer', names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>, index_col: 'IndexLabel | Literal[False] | None' = None, usecols=None, dtype: 'DtypeArg | None' = None, engine: 'CSVEngine | None' = None, converters=None, true_values=None, false_values=None, skipinitialspace: 'bool' = False, skiprows=None, skipfooter: 'int' = 0, nrows: 'int | None' = None, na_values=None, keep_default_na: 'bool' = True, na_filter: 'bool' = True, verbose: 'bool' = False, skip_blank_lines: 'bool' = True, parse_dates: 'bool | Sequence[Hashable] | None' = None, infer_datetime_format: 'bool | lib.NoDefault' = <no_default>, keep_date_col: 'bool' = False, date_parser=<no_default>, date_format: 'str

# Exercise: Short and long taxi rides

1. Load `taxi.csv` into a data frame. You want the following columns: `passenger_count`, `trip_distance`, `total_amount`, `tip_amount`.
2. What percentage of taxi riders never tip? (That is, zero tip?)
3. What percentage, on average, do they tip? (That is: All tips / all total amounts)

In [14]:
df = pd.read_csv('taxi.csv',
                usecols=['passenger_count', 'trip_distance', 'total_amount', 'tip_amount'])
df.head()

,passenger_count,trip_distance,tip_amount,total_amount
0,1,1.63,0.00,17.80
1,1,0.46,1.00,8.30
2,1,0.87,2.20,11.00
3,1,2.13,2.86,17.16
4,1,1.40,0.00,10.30


In [16]:
# what percentage of taxi riders never tip? tip is 0

# the value_counts method tallies up all of the values in a series, and tells us how many times each appears
(df['tip_amount'] == 0).value_counts()

tip_amount
False    5729
True     4270
Name: count, dtype: int64

In [18]:
# what percentage for each? 
# doesn't this strike you as a bit ... long?
(df['tip_amount'] == 0).value_counts() / df['tip_amount'].count()

tip_amount
False    0.572957
True     0.427043
Name: count, dtype: float64

In [19]:
# let's just get the percentages!
(df['tip_amount'] == 0).value_counts(normalize=True)

tip_amount
False    0.572957
True     0.427043
Name: proportion, dtype: float64

In [20]:
# what percentage do people tip, on average?

df['tip_amount'].sum() / df['total_amount'].sum()

0.1035785033739647

In [29]:
# want to know how many rows are in a data frame? Use len() on the index!
len(df.loc[df['tip_amount'] == 0].index)

4270

# Finding files

When I load a CSV file (or any other file) into Pandas, I can give it a few types of filenames. (I'm going to use Unix paths here, which use `/`. If you're on Windows, then just use `\\` instead, or a raw string with a single `\`.)

- If a filename contains no `/` characters, it's assumed to be in the current directory.
- If it contains a `/`, but doesn't start with one, then it's assumed to be in a subdirectory. For example, `abc/def.csv` is the `def.csv` file in the `abc` subdirectory.
- If it *starts* with a `/`, then it's an absolute path on your filesystem.

In this case, I just wrote `taxi.csv`, because the file is in the same directory as where I'm running Jupyter.

# What about files that are online?

We just saw that -- we can download `taxi.csv` onto our filesystem, and then load it with `pd.read_csv`.

What if we could shorten that?

You can pass a URL to read_csv, and it takes all of the other options that we've discussed.

In [31]:
pd.read_csv('https://raw.githubusercontent.com/reuven/oreilly-2023-05May-data/main/taxi.csv')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1,2015-06-01 00:12:59,2015-06-01 00:24:18,1,2.70,-73.947792,40.814972,1,N,-73.973358,40.783638,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30
9995,1,2015-06-01 00:12:59,2015-06-01 00:28:16,1,4.50,-74.004066,40.747818,1,N,-73.953758,40.779285,1,16.0,0.5,0.5,3.00,0.0,0.3,20.30
9996,2,2015-06-01 00:13:00,2015-06-01 00:37:25,1,5.59,-73.994377,40.766102,1,N,-73.903206,40.750546,2,21.0,0.5,0.5,0.00,0.0,0.3,22.30
9997,2,2015-06-01 00:13:02,2015-06-01 00:19:10,6,1.54,-73.978302,40.748531,1,N,-73.989166,40.762852,2,6.5,0.5,0.5,0.00,0.0,0.3,7.80


# Web scraping

I'ts not uncommon to find an HTML table on a web site with the data that we want. How can we retrieve the data from that table? Normally, we need "scrape" the web page, then grab the HTML table from within.

Pandas offers this to us automatically!  We just need to use the `pd.read_html` function.  Hand that a URL, and it returns a Python list of data frames. Every HTML table in the page is downloaded into a separate data frame.

You just need to identify which data frame is of interest, and then work on it.

In [32]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'

all_dfs = pd.read_html(url)

In [33]:
len(all_dfs)

7

In [36]:
all_dfs[2]

Country/Territory UN region IMF[1][13]            World Bank[14]   
    Country/Territory UN region   Estimate       Year       Estimate   
0               World         —  105568776       2023       96513077  \
1       United States  Americas   26854599       2023       22996100   
2               China      Asia   19373586  [n 1]2023       17734063   
3               Japan      Asia    4409738       2023        4937422   
4             Germany    Europe    4308854       2023        4223116   
..                ...       ...        ...        ...            ...   
212          Anguilla  Americas          —          —              —   
213          Kiribati   Oceania        248       2023            207   
214             Nauru   Oceania        151       2023            133   
215        Montserrat  Americas          —          —              —   
216            Tuvalu   Oceania         65       2023             63   

               United Nations[15]             
          Year           Estimate       Year  
0         2021           85328323       2020  
1         2021           23315081       2021  
2    [n 3]2021           17734131  [n 1]2021  
3         2021            4940878       2021  
4         2021            4259935       2021  
..         ...                ...        ...  
212          —                258       2020  
213       2021                181       2020  
214       2021                135       2020  
215          —                 68       2020  
216       2021                 55       2020  

[217 rows x 8 columns]

In [38]:
all_dfs[2]['IMF[1][13]']

,Estimate,Year
0,105568776,2023
1,26854599,2023
2,19373586,[n 1]2023
3,4409738,2023
4,4308854,2023
...,...,...
212,—,—
213,248,2023
214,151,2023
215,—,—


In [39]:
all_dfs[2]['IMF[1][13]'].dtypes

Estimate    object
Year        object
dtype: object

In [40]:
df = all_dfs[2]

In [41]:
df.dtypes

Country/Territory   Country/Territory    object
UN region           UN region            object
IMF[1][13]          Estimate             object
                    Year                 object
World Bank[14]      Estimate             object
                    Year                 object
United Nations[15]  Estimate             object
                    Year                 object
dtype: object

In [43]:
# I'll reassign the columns

df.columns = ['country', 'region', 'IMF estimate', 'IMF year', 'WB estimate', 'WB year', 'UN estimate', 'UN year']

In [44]:
df.head()

,country,region,IMF estimate,IMF year,WB estimate,WB year,UN estimate,UN year
0,World,—,105568776,2023,96513077,2021,85328323,2020
1,United States,Americas,26854599,2023,22996100,2021,23315081,2021
2,China,Asia,19373586,[n 1]2023,17734063,[n 3]2021,17734131,[n 1]2021
3,Japan,Asia,4409738,2023,4937422,2021,4940878,2021
4,Germany,Europe,4308854,2023,4223116,2021,4259935,2021


# Exercise: Grab financial news

1. Read all of the tables from the front page of https://finance.yahoo.com.
2. Find the "trending tickers" on that page
3. From that table, print all of the symbols and their latest prices.

# Other file formats

- what if I have a JSON file?
- what if I have an Excel file?

Pandas handles all of them! We can use `pd.read_excel` or `pd.read_json`, and read data into a Pandas data frame just fine.

In [47]:
url = 'https://gist.githubusercontent.com/reuven/77edbb0292901f35019f17edb9794358/raw/2bf258763cdddd704f8ffd3ea9a3e81d25e2c6f6/cities.json'

df = pd.read_json(url)

URLError: <urlopen error [Errno 54] Connection reset by peer>

In [48]:
!ls *.json

zsh:1: no matches found: *.json


# Next up:

1. Reading JSON
2. Sorting
3. Grouping

In [49]:
df = pd.read_json('cities.json')

In [50]:
!head -50 cities.json

[
    {
        "city": "New York", 
        "growth_from_2000_to_2013": "4.8%", 
        "latitude": 40.7127837, 
        "longitude": -74.0059413, 
        "population": "8405837", 
        "rank": "1", 
        "state": "New York"
    }, 
    {
        "city": "Los Angeles", 
        "growth_from_2000_to_2013": "4.8%", 
        "latitude": 34.0522342, 
        "longitude": -118.2436849, 
        "population": "3884307", 
        "rank": "2", 
        "state": "California"
    }, 
    {
        "city": "Chicago", 
        "growth_from_2000_to_2013": "-6.1%", 
        "latitude": 41.8781136, 
        "longitude": -87.6297982, 
        "population": "2718782", 
        "rank": "3", 
        "state": "Illinois"
    }, 
    {
        "city": "Houston", 
        "growth_from_2000_to_2013": "11.0%", 
        "latitude": 29.7604267, 
        "longitude": -95.3698028, 
        "population": "2195914", 
        "rank": "4", 
        "state": "Texas"
    }, 
    {
        "city": "Philadelphia

In [51]:
df.shape

(1000, 7)

In [52]:
df.head()

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
0,New York,4.8%,40.712784,-74.005941,8405837,1,New York
1,Los Angeles,4.8%,34.052234,-118.243685,3884307,2,California
2,Chicago,-6.1%,41.878114,-87.629798,2718782,3,Illinois
3,Houston,11.0%,29.760427,-95.369803,2195914,4,Texas
4,Philadelphia,2.6%,39.952584,-75.165222,1553165,5,Pennsylvania


# Sorting

Our data frame (of the cities, from the JSON file) is currently sorted in order of population rank, from highest to lowest. 

What if I want to see these cities in alphabetical order?

We can sort a series in two ways:
- by index (sort_index)
- by value (sort_values)

Here, we don't want to sort the individual column. We want to sort the entire data frame, based on the values in one of our columns. 

Fortunately, we have (more or less) the same methods available to us:

- sort_index -- which sorts the data frame based on the index
- sort_values -- but we have to choose which column to use

In [54]:
# this returns a new data frame, based on df, with the rows sorted by city name (ascending order)
df.sort_values('city')

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
220,Abilene,3.6%,32.448736,-99.733144,120099,221,Texas
985,Addison,2.6%,41.931696,-87.988956,37385,986,Illinois
115,Akron,-8.6%,41.081445,-81.519005,198100,116,Ohio
434,Alameda,5.4%,37.765206,-122.241636,76419,435,California
435,Albany,-0.6%,31.578507,-84.155741,76185,436,Georgia
...,...,...,...,...,...,...,...
836,York,6.4%,39.962598,-76.727745,43935,837,Pennsylvania
531,Youngstown,-20.2%,41.099780,-80.649519,65184,532,Ohio
529,Yuba City,27.9%,39.140448,-121.616911,65416,530,California
697,Yucaipa,26.8%,34.033625,-117.043087,52536,698,California


In [55]:
# reverse order, if you want
df.sort_values('city', ascending=False)

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
329,Yuma,16.2%,32.692651,-114.627692,91923,330,Arizona
697,Yucaipa,26.8%,34.033625,-117.043087,52536,698,California
529,Yuba City,27.9%,39.140448,-121.616911,65416,530,California
531,Youngstown,-20.2%,41.099780,-80.649519,65184,532,Ohio
836,York,6.4%,39.962598,-76.727745,43935,837,Pennsylvania
...,...,...,...,...,...,...,...
435,Albany,-0.6%,31.578507,-84.155741,76185,436,Georgia
434,Alameda,5.4%,37.765206,-122.241636,76419,435,California
115,Akron,-8.6%,41.081445,-81.519005,198100,116,Ohio
985,Addison,2.6%,41.931696,-87.988956,37385,986,Illinois


In [56]:
# sort by city name, and then assign back to df
df = df.sort_values('city')

In [57]:
df.head()

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
220,Abilene,3.6%,32.448736,-99.733144,120099,221,Texas
985,Addison,2.6%,41.931696,-87.988956,37385,986,Illinois
115,Akron,-8.6%,41.081445,-81.519005,198100,116,Ohio
434,Alameda,5.4%,37.765206,-122.241636,76419,435,California
435,Albany,-0.6%,31.578507,-84.155741,76185,436,Georgia


In [59]:
# we can sort by the index, also getting back a new data frame 
df.sort_index()

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
0,New York,4.8%,40.712784,-74.005941,8405837,1,New York
1,Los Angeles,4.8%,34.052234,-118.243685,3884307,2,California
2,Chicago,-6.1%,41.878114,-87.629798,2718782,3,Illinois
3,Houston,11.0%,29.760427,-95.369803,2195914,4,Texas
4,Philadelphia,2.6%,39.952584,-75.165222,1553165,5,Pennsylvania
...,...,...,...,...,...,...,...
995,Weslaco,28.8%,26.159519,-97.990837,37093,996,Texas
996,Keizer,14.4%,44.990119,-123.026208,37064,997,Oregon
997,Spanish Fork,78.1%,40.114955,-111.654923,36956,998,Utah
998,Beloit,2.9%,42.508348,-89.031776,36888,999,Wisconsin


In [66]:
# what if I want to sort by state, and then by city?
# in other words: sort first alphabetically by state, and then alphabetically by city name
# we can pass a list of columns, rather than a single column

df.sort_values(['state', 'city']).tail(10)

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
925,New Berlin,3.6%,42.976403,-88.108422,39834,926,Wisconsin
514,Oshkosh,5.3%,44.024706,-88.542614,66778,515,Wisconsin
420,Racine,-4.4%,42.726131,-87.782852,78199,421,Wisconsin
765,Sheboygan,-3.9%,43.750828,-87.714530,48725,766,Wisconsin
474,Waukesha,8.0%,43.011678,-88.231481,71016,475,Wisconsin
944,Wausau,1.7%,44.959135,-89.630122,39309,945,Wisconsin
794,Wauwatosa,0.0%,43.049457,-88.007587,47134,795,Wisconsin
581,West Allis,-0.6%,43.016681,-88.007031,60697,582,Wisconsin
598,Casper,19.9%,42.866632,-106.313081,59628,599,Wyoming
557,Cheyenne,16.9%,41.139981,-104.820246,62448,558,Wyoming


In [67]:
# what if I want to sort them first by state, and then by population?

df.sort_values(['state', 'population'])

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
982,Phenix City,31.9%,32.470976,-85.000765,37498,983,Alabama
921,Florence,10.2%,34.799810,-87.677251,40059,922,Alabama
810,Madison,53.7%,34.699258,-86.748332,45799,811,Alabama
652,Decatur,3.1%,34.605925,-86.983342,55816,653,Alabama
614,Auburn,26.4%,32.609857,-85.480782,58582,615,Alabama
...,...,...,...,...,...,...,...
271,Green Bay,1.9%,44.519159,-88.019826,104779,272,Wisconsin
82,Madison,15.8%,43.073052,-89.401230,243344,83,Wisconsin
30,Milwaukee,0.3%,43.038902,-87.906474,599164,31,Wisconsin
598,Casper,19.9%,42.866632,-106.313081,59628,599,Wyoming


In [68]:
# what if I want to sort first by state, and then by *decreasing* population?
# in other words: we want the states in alphabetical order
# but then, within each state, we want the cities in decreasing population order?

df.sort_values(['state', 'population'],    # these are the columns to sort by
               ascending=[True, False])    # for each column - -ascending or not?

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
100,Birmingham,-12.3%,33.520661,-86.802490,212113,101,Alabama
110,Montgomery,-0.1%,32.366805,-86.299969,201332,111,Alabama
121,Mobile,-1.9%,30.695366,-88.039891,194899,122,Alabama
125,Huntsville,16.3%,34.730369,-86.586104,186254,126,Alabama
312,Tuscaloosa,21.1%,33.209841,-87.569173,95334,313,Alabama
...,...,...,...,...,...,...,...
970,Brookfield,-1.9%,43.060567,-88.106479,37999,971,Wisconsin
992,Greenfield,4.8%,42.961404,-88.012586,37159,993,Wisconsin
998,Beloit,2.9%,42.508348,-89.031776,36888,999,Wisconsin
557,Cheyenne,16.9%,41.139981,-104.820246,62448,558,Wyoming


# Exercise: Sorted taxis

1. Sort the rides in order of `total_amount`. What were the 10 less expensive taxi rides, and what were the 10 most expensive taxi rides? In both of these cases, how much did people pay per mile?
2. Sort the taxi rides in order, first by `tpep_pickup_datetime` (ascending) and then by `trip_distance` (descending).

In [85]:
df = pd.read_csv('taxi.csv',
                usecols=['tpep_pickup_datetime',
                         'passenger_count', 'trip_distance', 'total_amount', 'tip_amount'])
df.head()

,tpep_pickup_datetime,passenger_count,trip_distance,tip_amount,total_amount
0,2015-06-02 11:19:29,1,1.63,0.00,17.80
1,2015-06-02 11:19:30,1,0.46,1.00,8.30
2,2015-06-02 11:19:31,1,0.87,2.20,11.00
3,2015-06-02 11:19:31,1,2.13,2.86,17.16
4,2015-06-02 11:19:32,1,1.40,0.00,10.30


In [86]:
cheapest_rides = df.sort_values('total_amount').head(10)

cheapest_rides['total_amount'].sum() / cheapest_rides['trip_distance'].sum()

-1.596385542168675

In [87]:
expensive_rides = df.sort_values('total_amount').tail(10)

expensive_rides['total_amount'].sum() / expensive_rides['trip_distance'].sum()

6.456473015760077

In [88]:
# Sort the taxi rides in order, first by tpep_pickup_datetime (ascending) and then by trip_distance (descending).

df.sort_values(['tpep_pickup_datetime', 'trip_distance'],
              ascending=[True, False])

,tpep_pickup_datetime,passenger_count,trip_distance,tip_amount,total_amount
4063,2015-06-01 00:00:00,2,1.40,0.00,12.30
3987,2015-06-01 00:00:00,1,1.00,0.00,7.30
4103,2015-06-01 00:00:00,1,0.90,2.00,15.30
4025,2015-06-01 00:00:01,1,8.15,2.50,30.30
4159,2015-06-01 00:00:01,1,7.41,5.57,33.41
...,...,...,...,...,...
8163,2015-06-06 16:53:56,1,1.30,1.45,8.75
8162,2015-06-06 16:53:56,1,1.20,2.19,9.49
8087,2015-06-06 16:53:56,1,0.76,0.00,4.80
8202,2015-06-06 16:53:56,1,0.58,0.95,5.75


 # Grouping
 
 I might want to know if there is a substantial difference in how far passengers go in taxis, depending on how many passengers there are.
 
Fortunately, I know how to ask and answer this question.

In [93]:
# mean distance traveled by 1 person
df.loc[df['passenger_count'] == 1, 'trip_distance'].mean()

3.0923380047176354

In [94]:
# mean distance traveled by 2 people
df.loc[df['passenger_count'] == 2, 'trip_distance'].mean()

3.3843869002284848

In [95]:
# mean distance traveled by 3 people
df.loc[df['passenger_count'] == 3, 'trip_distance'].mean()

3.3423891625615765

This kind of query is very usual:

- Per product, how much are we making, on average?
- How many people are there in each department?
- What are the average salaries paid in each department?

What we see here is an attempt to run our aggregation methods, but instead of running them against the entire data set, we want to run them once per value of a categorical column.

In other words: Find a categorical column. Find a numeric column.  For each value of the categorical, calculate the aggregation method on the numeric column.

In [96]:
# the 'groupby' method does this

# we'll need:
# - categorical to group on -- passenger_count
# - a numeric column to calculate on -- trip_distance
# - an aggregation method to perform the calculation -- mean

df.groupby('passenger_count')['trip_distance'].mean()

passenger_count
0    4.600000
1    3.092338
2    3.384387
3    3.342389
4    3.628901
5    3.182712
6    3.170976
Name: trip_distance, dtype: float64

# Don't use non-categorical numerical columns for grouping!

Pandas will allow you to group by anything. And here, we grouped by passenger count, which is numeric, but we know that it's limited in scope.

If you try to group by `trip_distance`, the results will be nonsensical, and the series/data frame you get back from the groupby will be huge.

In [97]:
df.groupby('trip_distance')['total_amount'].mean()

trip_distance
0.00      31.58194
0.01      52.80000
0.02      43.46000
0.03       3.96000
0.04      70.01000
           ...    
34.84    137.59000
35.51    135.13000
37.20    210.14000
60.30    160.05000
64.60     79.96000
Name: total_amount, Length: 1219, dtype: float64

In [98]:
!head airlines.dat

1,"Private flight",\N,"-","N/A","","","Y" 
2,"135 Airways",\N,"","GNL","GENERAL","United States","N"
3,"1Time Airline",\N,"1T","RNX","NEXTIME","South Africa","Y"
4,"2 Sqn No 1 Elementary Flying Training School",\N,"","WYT","","United Kingdom","N"
5,"213 Flight Unit",\N,"","TFU","","Russia","N"
6,"223 Flight Unit State Airline",\N,"","CHD","CHKALOVSK-AVIA","Russia","N"
7,"224th Flight Unit",\N,"","TTF","CARGO UNIT","Russia","N"
8,"247 Jet Ltd",\N,"","TWF","CLOUD RUNNER","United Kingdom","N"
9,"3D Aviation",\N,"","SEC","SECUREX","United States","N"
10,"40-Mile Air",\N,"Q5","MLA","MILE-AIR","United States","Y"


# Exercise: Airlines per country

1. `airlines.dat` is a CSV file containing information about all the airlines in the world (Or so I think).
2. It has no header, as you can see, but fields are separated by commas.
3. We're only interested in the name of the airline (field 5) and the country the airline is from (field 6).
4. Load the data into a data frame.
5. Use the `count` aggregation method to find the number of airlines in each country. Which 10 countries have the most airlines?

In [103]:
# easiest way to do this
df = pd.read_csv('airlines.dat',
                header=None,
                usecols=[5,6])
df.columns = ['name', 'country']

df.head()

,name,country
0,NaN,NaN
1,GENERAL,United States
2,NEXTIME,South Africa
3,NaN,United Kingdom
4,NaN,Russia


In [104]:
# fanciest way to do this
df = pd.read_csv('airlines.dat',
                header=None,
                usecols=[5,6],
                names=['name', 'country'])

df.head()

,name,country
0,NaN,NaN
1,GENERAL,United States
2,NEXTIME,South Africa
3,NaN,United Kingdom
4,NaN,Russia


In [105]:
# categorical : country
# calculate on : name
# aggregation method: count

df.groupby('country')['name'].count()

country
 Boonville Stage Line     1
 S.A.                     1
ACOM                      1
ACTIVE AERO               1
AEROCENTER                1
                         ..
WATCHDOG                  1
Yemen                     1
Zambia                   20
Zimbabwe                  6
\N                        3
Name: name, Length: 276, dtype: int64

In [107]:
df.groupby('country')['name'].count().sort_values(ascending=False).head(10)

country
United States     969
Mexico            429
United Kingdom    358
Canada            293
Russia            179
Spain             152
Germany           113
France            104
Ukraine            85
South Africa       85
Name: name, dtype: int64

In [108]:
(
    df.groupby('country')['name'].     # for each value of country, we'll count how many names there are
    count().
    sort_values(ascending=False).      # take the result of grouping and sort the values (airline counts)
    head(10)                           # find the 10 countries with the greatest number of airlines
)

country
United States     969
Mexico            429
United Kingdom    358
Canada            293
Russia            179
Spain             152
Germany           113
France            104
Ukraine            85
South Africa       85
Name: name, dtype: int64

# Aggregation methods

- `count`
- `sum`
- `min`
- `max`
- `mean`
- `std`
- `median`


# What if we want to calculate based on more than one column?

As usual, in Pandas, the rule of thumb is that anywhere you can pass a string (column name), you can pass a list of strings (multiple column names).



In [109]:
# fanciest way to do this
df = pd.read_csv('airlines.dat',
                header=None,
                usecols=[1, 5,6],
                names=['longname', 'name', 'country'])

df.head()




,longname,name,country
0,Private flight,NaN,NaN
1,135 Airways,GENERAL,United States
2,1Time Airline,NEXTIME,South Africa
3,2 Sqn No 1 Elementary Flying Training School,NaN,United Kingdom
4,213 Flight Unit,NaN,Russia


In [112]:
# we always need to put the name of the column we're calculating on in []
# if we have more than one name, we need to use [[]]

# count only counts non-NaN values -- so you'll often get different counts per column

df.groupby('country')[['longname', 'name']].count().sort_values('longname', ascending=False)

,longname,name
country,,
United States,1080,969
Mexico,439,429
United Kingdom,407,358
Canada,318,293
Russia,230,179
...,...,...
AVIANCA,1,1
Russia]],1,1
AVINOR,1,1


In [120]:
# if you give names to all columns, then you can pass names to usecols
# otherwise, you need to pass integers for usecols, and name the columns you want

df = pd.read_csv('airlines.dat',
                header=None,
                names=['a', 'longname', 'b', 'c', 'd', 'name', 'country', 'e'],
                usecols=['longname', 'name', 'country'])

df.head()


,longname,name,country
0,Private flight,NaN,NaN
1,135 Airways,GENERAL,United States
2,1Time Airline,NEXTIME,South Africa
3,2 Sqn No 1 Elementary Flying Training School,NaN,United Kingdom
4,213 Flight Unit,NaN,Russia


# Next up

- Grouping by multiple columns
- Multiple aggregation methods
- Advanced grouping techniques
- Pivot tables

In [122]:
df = pd.read_csv('taxi.csv', usecols=['trip_distance', 'total_amount', 'passenger_count'])

In [123]:
# for each value of passenger_count, find the mean trip_distance and total_amount
df.groupby('passenger_count')[['trip_distance', 'total_amount']].mean()

,trip_distance,total_amount
passenger_count,,
0,4.600000,25.570000
1,3.092338,17.368569
2,3.384387,18.406306
3,3.342389,17.994704
4,3.628901,18.881648
5,3.182712,17.211269
6,3.170976,17.401355


In [126]:
# what if I want to run more than one aggregation method?
# that is: What if I want to get both the mean and the standard deviation
#   for trip_distance, per value in passenger_count?

# group on (categorical): passenger_count
# aggregation method: mean + standard deviation
# calculate on: trip_distance

# we can use the ".agg" method to provide more than one aggregation method
# we can pass a list of either strings (uniquely describing the methods) or
# the methods themselves

df.groupby('passenger_count')['trip_distance'].agg(['mean', 'std'])

,mean,std
passenger_count,,
0,4.600000,4.666905
1,3.092338,4.020187
2,3.384387,4.242826
3,3.342389,3.822041
4,3.628901,4.351369
5,3.182712,3.969468
6,3.170976,3.759807


In [127]:
df.groupby('passenger_count')['trip_distance'].agg([np.mean, np.std])

,mean,std
passenger_count,,
0,4.600000,4.666905
1,3.092338,4.020187
2,3.384387,4.242826
3,3.342389,3.822041
4,3.628901,4.351369
5,3.182712,3.969468
6,3.170976,3.759807


In [128]:
# we've seen that we can calculate on one or more columns
# we can invoke one or more aggregation methods



In [131]:
!head olympic_athlete_events.csv

# Exercise: Olympic athlete groupings

1. Load the file (now in GitHub) called `olympic_athlete_events.csv`. This file contains info about every athlete, from every team, from every Olympic games.
2. For each value of `Sex`, what are the mean `Height` and `Weight` of athletes?
3. For each year, what were the mean and standard deviation ages?